In [1]:
%%capture
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, AutoModelForCausalLM ,GPT2Model
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from torchvision import transforms
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from google.colab import files
import os
import numpy as np
model_name = "HooshvareLab/gpt2-fa"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
print(tokenizer.encode("<sep>"))

[9]


(آ)

In [4]:
A = []
B = []
# Open the file
with open('ferdousi.txt', 'r') as file:
    # Enumerate the lines, line_num is the line number and line is the line itself
    for line_num, line in enumerate(file, start=0):
        # If the line number is odd
        if line_num % 2 != 1:
            A.append(line.strip()+'<sep>')
        # If the line number is even
        else:
            B.append(line.strip()+'<sep>')

# Now, 'A' contains the odd lines and 'B' contains the even lines
A ,B= A[1:], B[1:]


print(B[0])
print(A[0])
print('num of verses:',len(A))
print('num of verses:',len(B))

کزین برتر اندیشه برنگذرد<sep>
به نام خداوند جان و خرد<sep>
num of verses: 49608
num of verses: 49608


In [5]:
input_tokenized = tokenizer(A, return_tensors='pt', padding=True, truncation=False)
target_tokenized = tokenizer(B, return_tensors='pt', padding=True, truncation=False)

In [6]:
data_input = input_tokenized['input_ids']
data_target =target_tokenized['input_ids']
attention_input = input_tokenized['attention_mask']
attention_target = target_tokenized['attention_mask']

input_train, input_test, target_train, target_test, input_attention_train , input_attention_test    = train_test_split(data_input,
                                                                      data_target, attention_input,test_size=0.1, random_state=42)

In [7]:
vers1 = tokenizer.decode(input_train[0], skip_special_tokens=True)
vers2 = tokenizer.decode(target_train[0], skip_special_tokens=True)
print(vers1)
print(vers2)

وگر در میان دو رویه سپاه <sep>
بگردی بلاف از پی نام و جاه <sep>


In [8]:
class CustumeDataset(Dataset):
    def __init__(self, input_ids, attention_mask_input, target_ids):
        self.input_ids = input_ids
        self.target_ids = target_ids
        self.attention_mask_input = attention_mask_input

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask_input[idx], self.target_ids[idx]

# Create dataset
train_set = CustumeDataset(input_train, input_attention_train, target_train)
test_set = CustumeDataset(input_test, input_attention_test, target_test)

# Create dataloader
trainloader = DataLoader(train_set, batch_size=32, shuffle=True,num_workers =2)
testloader = DataLoader(test_set, batch_size=32, shuffle=True,num_workers =2)

In [21]:
%%capture
config = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=config)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

In [10]:
for data,attention,target in trainloader:
  data = data.to(device)
  attention = attention.to(device)
  target = target.to(device)
  break

In [11]:
output = model.generate(data,max_length=26)
generated_text = tokenizer.decode(output[10], skip_special_tokens=True)
input_text = tokenizer.decode(data[10], skip_special_tokens=True)
print((input_text))
print(generated_text[len(input_text):])


ز من باد بر شاه ایران درود <sep>
  است که در آن از دو واژهٔ «م» و «


In [12]:
def val_loop(testloader,model,loss_function):
  model.eval()
  with torch.no_grad():
    for i, (inputs,attention, targets) in enumerate(testloader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        attention = attention.to(device)

        # Forward pass
        outputs = model(inputs,attention_mask=attention, labels=targets)
        logits = outputs.logits

        # Reshape the logits and targets and compute the loss
        return loss_function(logits.view(-1, logits.size(-1)), targets.view(-1))


In [22]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters() , lr= 0.001 , eps= 1e-8 )
min_val_loss = float('inf')
# Number of training epochs
epochs = 50
n = 0
itter = 0
val_loss = []
mean_val_losses = []
# Training loop
for epoch in range(epochs):
    running_loss = 0
    model.train()

    for (inputs,attention, targets) in tqdm(trainloader):
        itter += 1
        inputs = inputs.to(device)
        targets = targets.to(device)
        attention = attention.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs,attention_mask=attention, labels=targets)
        logits = outputs.logits

        # Reshape the logits and targets and compute the loss
        loss = loss_function(logits.view(-1, logits.size(-1)), targets.view(-1))

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Evaluation
        if itter % 100 == 0:
            val_loss.append(val_loop(testloader,model,loss_function).item())
            if len(val_loss)<10:
              val_loss = 10*val_loss

            mean_val_loss = np.mean(val_loss[-10:])
            mean_val_losses.append(mean_val_loss)

            if len(mean_val_losses) > 1:
                if mean_val_losses[-2] > mean_val_losses[-1]:
                  n = 0
                  # Save the model
                  checkpoint = {'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict()}

                elif mean_val_losses[-2] <= mean_val_losses[-1]:
                    n+=1
                    print(f"Validation loss is increasing:{val_loss[-1]}")
                if n == 3 :
                  model.load_state_dict(checkpoint['model_state_dict'])
                  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                  break
    if n == 3:
      break
    print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss/len(trainloader):.4f}, Validation Loss: {val_loss[-1]/len(testloader):.4f}')
torch.save(checkpoint, "checkpoint.pth")
    # if val_loss < min_val_loss:
    #     # Save the model
    #     checkpoint = {'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict()}
    #     torch.save(checkpoint, "checkpoint.pth")
    #     min_val_loss = val_loss
    #     n = 0
    # elif n == 2:
    #   model.load_state_dict(checkpoint['model_state_dict'])
    #   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #   break
    # else: n +=1

# files.download('checkpoint.pth')


100%|██████████| 1396/1396 [03:37<00:00,  6.41it/s]


Epoch [1/50], Training Loss: 3.7427, Validation Loss: 0.0237


  0%|          | 5/1396 [00:01<05:14,  4.42it/s]

Validation loss is increasing:3.664842128753662


 58%|█████▊    | 805/1396 [02:05<01:49,  5.41it/s]

...

Validation loss is increasing:3.647052526473999


 15%|█▍        | 209/1396 [00:33<03:58,  4.97it/s]

Validation loss is increasing:3.6896321773529053


 22%|██▏       | 307/1396 [00:48<02:53,  6.27it/s]

Validation loss is increasing:3.7954695224761963


In [14]:
# if 'checkpoint.pth' in os.listdir():
#   checkpoint = torch.load('checkpoint.pth')
#   model.load_state_dict(checkpoint['model_state_dict'])
#   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#   print('pre_train model is exist')
# torch.cuda.empty_cache()

In [23]:
for i, (inputs,attention, targets) in enumerate(trainloader):
   inputs = inputs.to(device)
   targets = targets.to(device)
   attention = attention.to(device)
   break

In [24]:
print('Generating for training data')
generated_text = model.generate(inputs,max_length=26,
        temperature=0.7,num_beams=10,
        no_repeat_ngram_size=2,
        top_k=50,top_p=0.95,
        pad_token_id=tokenizer.pad_token_id)
for j in range(20):
        input_text = tokenizer.decode(inputs[j], skip_special_tokens=True)
        output_text = tokenizer.decode(generated_text[j], skip_special_tokens=True)
        print(f"Input{j}:     {input_text.replace('<sep>','')}")
        print(f"Generated{j}: {output_text.replace('<sep>','')[0:]}")

Generating for training data
Input0:     همم گنج و بوم است و هم چارپای 
Generated0: همم گنج و بوم است و هم چارپای     گاه   و   به و و به 
Input1:     بریزم ز تن خون انباردار 
Generated1: بریزم ز تن خون انباردار   خوار   گار و   کارزار   خوار
Input2:     نهادند سوی فرامرز روی 
Generated2: نهادند سوی فرامرز روی    و و    گاه می نه و و
Input3:     ازان تازی اسپان کش آمد گزین 
Generated3: ازان تازی اسپان کش آمد گزین   چین    چین   کین کین   زمین زمین 
Input4:     پیاده بیامد به نزدیک شاه 
Generated4: پیاده بیامد به نزدیک شاه     گاه گاه   راه راه و   سپاه 
Input5:     به تیزی بیامد به نزدیک شاه 
Generated5: به تیزی بیامد به نزدیک شاه   گاه    کلاه راه   گاه   راه به
Input6:     خدای جهان را نباشد نیاز 
Generated6: خدای جهان را نباشد نیاز    نیاز نیاز و گاه باز   باز نیاز
Input7:     کسی را ندانم که روز نبرد 
Generated7: کسی را ندانم که روز نبرد   گرد    گرد  ژ و   کرد نبرد کرد
Input8:     بر رستم آمد همانگاه گیو 
Generated8: بر رستم آمد همانگاه گیو   نیو   و و   نیو   گی گی
Input9:     به

In [25]:
for i, (inputs,attention, targets) in enumerate(testloader):
   inputs = inputs.to(device)
   targets = targets.to(device)
   attention = attention.to(device)
   break

In [26]:
print('Generating for test data')
generated_text = model.generate(inputs,max_length=26,
        temperature=0.7,num_beams=10,
        no_repeat_ngram_size=2,
        top_k=20,top_p=0.95,
        pad_token_id=tokenizer.pad_token_id)
for j in range(generated_text.shape[0]):
        input_text = tokenizer.decode(inputs[j], skip_special_tokens=True)
        output_text = tokenizer.decode(generated_text[j], skip_special_tokens=True)
        print(f"Input{j}:     {input_text.replace('<sep>','')}")
        print(f"Generated{j}: {output_text.replace('<sep>','')[0:]}")

Generating for test data
Input0:     نباید که در پیش خسرو شود 
Generated0: نباید که در پیش خسرو شود   واندواندواند  شود شودواند نماند  
Input1:     همان به که او را برپهلوان 
Generated1: همان به که او را برپهلوان   و    و   دل به بر   بک
Input2:     و گر شاه و فرزانگان این به جای 
Generated2: و گر شاه و فرزانگان این به جای     پایمای   و و   رهن  مای
Input3:     درفش سرافراز خاقان و تاج 
Generated3: درفش سرافراز خاقان و تاج    عاج گاه عاج و   تاج تاج تاج عاج
Input4:     بیامد چو نزدیک ایشان رسید 
Generated4: بیامد چو نزدیک ایشان رسید    دید دید   کشید   بدید   برکشید
Input5:     چنان بد که روزی به نخچیرگاه 
Generated5: چنان بد که روزی به نخچیرگاه     گاه شاه راه سپاه و   و
Input6:     چو گشتاسپ را دید بر تخت عاج 
Generated6: چو گشتاسپ را دید بر تخت عاج   تاج    عاج عاج و تاج   تاج تاج عاج
Input7:     بیاراست پیلان و برخاست غو 
Generated7: بیاراست پیلان و برخاست غو   و    تو نو   نو راست   و
Input8:     ز طایر یکی دختش آمد چو ماه 
Generated8: ز طایر یکی دختش آمد چو ماه   و    آمد بودش ک